In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2

2021-12-05 20:51:52.731634: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-05 20:51:52.731653: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [71]:
_map = {3: "leftEar", 4: "rightEar", 5: "leftShoulder", 6: "rightShoulder", 7: "leftElbow", 8: "rightElbow", 9: "leftWrist", 
        10: "rightWrist", 11: "leftHip", 12: "rightHip", 13: "leftKnee", 14: "rightKnee", 15: "leftAnkle", 16: "rightAnkle"}

In [3]:
interpreter = tf.lite.Interpreter(model_path="../models/lite-model_movenet_singlepose_lightning_3.tflite")
interpreter.allocate_tensors()

In [81]:
def draw_keypoints(frame, keypoint, confidence):
    font = cv2.FONT_HERSHEY_SIMPLEX
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoint,[y, x, 1]))
    
    # Avoid eyes and nose tracking
    index = 3
    for kp in shaped[3:]:
        ky, kx, kp_conf = kp
        if kp_conf > confidence:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0, 255, 0), -1)
            # Add the probability and the name of the point
            cv2.putText(frame, _map[index] + ": " + str(round(kp_conf, 2)), (int(kx), int(ky)), font, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        index += 1

In [86]:
## Webcam
#cap = cv2.VideoCapture(0)

## Video
cap = cv2.VideoCapture('../rowing2.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192, 192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    interpreter.set_tensor(input_details[0]["index"], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]["index"])
    
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    
    cv2.imshow("Movenet test", frame)
    if cv2.waitKey(10) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()